In [1]:
pip install requests beautifulsoup4 mysql-connector-python


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [ ]:
CREATE DATABASE website_data;
USE website_data;

CREATE TABLE website_titles (
    id INT AUTO_INCREMENT PRIMARY KEY,
    website_name VARCHAR(255),
    title_content VARCHAR(255)
);


In [ ]:
import requests
from bs4 import BeautifulSoup
import mysql.connector

# Function to extract title content from the given website
def get_website_title(website_url):
    try:
        # Send HTTP request to the website
        response = requests.get(website_url)
        response.raise_for_status()  # Check if the request was successful
        
        # Parse the content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the title tag and extract its content
        title_tag = soup.find('title')
        title_content = title_tag.text if title_tag else "No title found"
        
        return title_content
    except Exception as e:
        print(f"Error: {e}")
        return None

# Function to save website and title content to MySQL
def save_to_mysql(website_name, title_content):
    try:
        # Connect to MySQL database
        conn = mysql.connector.connect(
            host='localhost',
            user='root',  # Replace with your MySQL username
            password='your_password',  # Replace with your MySQL password
            database='website_data'  # Replace with your database name
        )
        cursor = conn.cursor()
        
        # Insert website name and title content into the table
        query = "INSERT INTO website_titles (website_name, title_content) VALUES (%s, %s)"
        cursor.execute(query, (website_name, title_content))
        
        # Commit the transaction
        conn.commit()
        print("Data successfully inserted into the database.")
        
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        cursor.close()
        conn.close()

# Main function
def main():
    # Accept website URL from the user
    website_url = input("Enter the website URL (e.g., https://www.example.com): ")
    
    # Extract the website name (domain)
    website_name = website_url.split("//")[-1].split("/")[0]
    
    # Get the title content
    title_content = get_website_title(website_url)
    
    if title_content:
        # Save the website name and title content to MySQL
        save_to_mysql(website_name, title_content)
    else:
        print("Failed to extract title content.")

if __name__ == "__main__":
    main()
